# Development of miscellaneous new features for polymerist

## Testing polymerist importability within environment

In [ ]:
import polymerist as ps

print(ps.__version__)
print(ps.pascal(5))

In [ ]:
from polymerist.mdtools import openfftools

openfftools.POLYMERIST_TOOLKIT_REGISTRY.registered_toolkits

In [ ]:
import polymerist as ps
from polymerist.genutils.importutils import pyimports, module_hierarchy

print(module_hierarchy(ps))

In [ ]:
import nglview

print(nglview.__version__)
nglview.demo()

In [ ]:
from polymerist.polymers.monomers import specification
from rdkit import Chem

smi = 'CCO-c1ccccc1-N=C=C'
mol1 = Chem.MolFromSmiles(smi)
display(mol1)

sma = specification.expanded_SMILES(smi, assign_map_nums=True)
exp_sma = specification.compliant_mol_SMARTS(sma)
mol2 = Chem.MolFromSmarts(sma)
display(mol2)


In [ ]:
import numpy as np
import pandas as pd

from openff.toolkit import Molecule, Topology, ForceField

offmol = Molecule.from_smiles(smi)
offmol.generate_conformers(n_conformers=1)
offmol.visualize(backend='nglview')

# Testing kekulization flag in expanded_SMILES and rdkdraw

In [4]:
from rdkit import Chem

from polymerist.polymers.monomers.specification import expanded_SMILES
from polymerist.rdutils import set_rdkdraw_size, enable_kekulized_drawing, disable_kekulized_drawing

In [ ]:
disable_kekulized_drawing()
# enable_kekulized_drawing()

smi = 'c1ccccc1C(=O)O'
exp_smi = expanded_SMILES(smi, kekulize=False)
mol = Chem.MolFromSmiles(exp_smi, sanitize=False)

display(mol)
for bond in mol.GetBonds():
    print(bond.GetBondType())

## Experimenting with Protocols

In [ ]:
from typing import Protocol, runtime_checkable
from dataclasses import dataclass


@runtime_checkable
class Nominative(Protocol):
    def name(self) -> str:
        ...
        
class Foo(Nominative):
    def __init__(self, letter : str):
        self.letter = letter
        
    def name(self) -> str:
        return self.letter*3

@dataclass
class Bar:
    id : int
    
    def name(self) -> str:
        return f'{type(self).__name__}{self.id}'
    
@dataclass
class Baz:
    id : int
    
    def nombre(self) -> str:
        return 'name'

In [ ]:
classes = (Foo, Bar, Baz)
a = Foo('a')
b = Bar(2)
c = Baz(3)
instances = (a, b, c)

In [ ]:
for typ in classes:
    print(typ, issubclass(typ, Nominative))

In [ ]:
for inst in instances:
    print(inst, isinstance(inst, Nominative))

# Parsing lines from PDB file

In [ ]:
l1 = 'ATOM    189  C99 OCT     5      39.590  30.100  38.320  1.00  0.00'           
l2 = 'ATOM    190 C100 OCT     5      38.850  31.110  37.700  1.00  0.00'
l3 = 'HETATM   47  H21 UNL     1       0.000   0.000   0.000  1.00  0.00           H '

In [ ]:
from typing import Any

PDB_ATOM_TOKEN_COLUMNS : dict[str, tuple[int, int]] = {
    'Is Heteratom' : (1, 6),
    'Atom serial number' : (7, 11),
    'Atom name' : (13, 16),
    'Alternate location indicator' : (17, 17),
    'Residue name' : (18, 20),
    'Chain identifier' : (22, 22),
    'Residue sequence number' : (23, 26),
    'Code for insertions of residues' : (27, 27),
    'X (angstrom)' : (31, 38),
    'Y (angstrom)' : (39, 46),
    'Z (angstrom)' : (47, 54),
    'Occupancy' : (55, 60),
    'Temperature factor' : (61, 66),
    'Segment identifier' : (73, 76),
    'Element symbol' : (77, 78),
    'Charge' : (79, 80),
} # taken from PDB spec doc (https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html)
def tokenize_pdb_atom_line(line : str) -> dict[str, Any]:
    line = line.ljust(80, ' ') # pad line to 80 characters with spaces
    return {
        prop_name : line[i_start-1:i_end]
            for prop_name, (i_start, i_end) in PDB_ATOM_TOKEN_COLUMNS.items()
    }


for line in (l1, l2, l3):
    print(tokenize_pdb_atom_line(line))

# Another thing